In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libraries
import os
import pickle

from src.models import train_model, test_model, predict_model

In [3]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [4]:
EXPERIMENT = 'mh15'
MODEL = 'model_A2'
EXP_DIR = f'./models/{EXPERIMENT}/'
# Training
PATH_TO_DATA = f'data/processed/synthetic_data/model_A1/n1000/'
# Models
model_template = f'./models/templates/{MODEL}_noisyor.pl'

Training

In [5]:
train_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_DATA, 
    model_template = model_template, 
    training = 0.8, 
    max_iter = 20, 
    evidence = ['activity', 'occupancy', 'fold_change'],
    facts = [],
    suffix = ''
)

Splitting data...
Starting LFI...
problog lfi ./models/templates/model_A2_noisyor.pl ./models/mh15/evidence.pl -O ./models/mh15/model_trained.pl --normalize -k ddnnf -v -n 20
[INFO] Output level: INFO
[INFO] Number of examples: 800
[INFO] Compile time: 53.758343
[INFO] Weights to learn: [t(0.33,P)::fold_change(P,S,inc), t(0.33,P)::fold_change(P,S,dec), t(0.33,P)::fold_change(P,S,baseline), t(0.33,E)::activity(E,S,inc), t(0.33,E)::activity(E,S,dec), t(0.33,E)::activity(E,S,baseline), t(0.33,_fc)::occupancy(P,S,inc), t(0.33,_fc)::occupancy(P,S,dec), t(0.33,_fc)::occupancy(P,S,baseline), t(0.33,E,_dM)::occupancy(P,S,inc), t(0.33,E,_dM)::occupancy(P,S,dec), t(0.33,E,_dM)::occupancy(P,S,baseline)]
[INFO] Bodies: [lfi_body(0,t(P),P,S,inc), lfi_body(1,t(P),P,S,dec), lfi_body(2,t(P),P,S,baseline), lfi_body(3,t(E),E,S,inc), lfi_body(4,t(E),E,S,dec), lfi_body(5,t(E),E,S,baseline), lfi_body(6,t(_fc),P,S,inc), lfi_body(7,t(_fc),P,S,dec), lfi_body(8,t(_fc),P,S,baseline), lfi_body(9,t(E,_dM),P,S,inc

Testing

In [6]:
# Testing
model_trained = f'{EXP_DIR}model_trained.pl'
test = ['training', 'testing']
PATH_TO_TRUTH = f'data/processed/synthetic_data/model_A1/n1000/'
results = {}

In [7]:
for test_on in test:
    PATH_TO_TESTING = f'{EXP_DIR}{test_on}/'
    compare_to = [test_on, 'truth']
    for compare in compare_to:
        if compare == 'truth':
            path_to_truth = PATH_TO_TRUTH
        else: 
            path_to_truth = PATH_TO_TESTING

        test_results = test_model.main(
            experiment = EXPERIMENT, 
            path_to_data = PATH_TO_TESTING, 
            path_to_truth = path_to_truth, 
            model_trained = model_trained, 
            targets = ['occupancy', 'modification'], 
            test_on = test_on, compare = compare, 
            mode = 'inference')
        results[f'{test_on}_vs_{compare}'] = test_results

Inferring data (s1)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s1)...
Inferring data (s1000)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s1000)...
Inferring data (s102)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s102)...
Inferring data (s103)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s103)...
Inferring data (s104)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s104)...
Inferring data (s105)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s105)...
Inferring data (s106)...
problog ./models/mh15/test_on_training.pl -o ./models/mh15/predictions/inferred_example.txt
Finished 

In [8]:
PATH_TO_TESTING = f'{EXP_DIR}testing/'
predictions = predict_model.main(
    experiment = EXPERIMENT, 
    path_to_data = PATH_TO_TESTING, 
    model_trained = model_trained, 
    targets = ['occupancy', 'modification'], 
    mode = 'inference', infer_from = 'facts',
    suffix = '_testing')
results['predictions'] = predictions

Inferring data (s10)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s10)...
Inferring data (s100)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s100)...
Inferring data (s101)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s101)...
Inferring data (s109)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s109)...
Inferring data (s117)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s117)...
Inferring data (s12)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished inference (s12)...
Inferring data (s126)...
problog ./models/mh15/make_predictions.pl -o ./models/mh15/predictions/inferred_example.txt
Finished in

In [9]:
with open(f'{EXP_DIR}results.pickle', 'wb') as file:
    pickle.dump(results, file, protocol=pickle.HIGHEST_PROTOCOL)